# Data preprocess

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D
import cv2
import os

# parameters
NUM_EACH_SUBJECT = 170
NUM_SELFIES = 10
TRAIN_TEST_RATIO = 0.7
RANDOM_SEED = 22
np.random.seed(RANDOM_SEED)

def choose_random_idx(num, vmin, vmax, seed=RANDOM_SEED):
    np.random.seed(seed)
    idx = [i for i in range(vmin, vmax)]
    random_idx = np.random.permutation(idx)
    return sorted(random_idx[0 : num])

def get_train_test_list(input_list, ratio, seed=RANDOM_SEED):
    train_idx = choose_random_idx(num=round(len(input_list)*ratio), vmin=0, vmax=len(input_list), seed=RANDOM_SEED)

    train_list = []
    test_list = []
    for i in range(0,len(input_list)):
        if i in train_idx:
            train_list.append(input_list[i])
        else:
            test_list.append(input_list[i])
    return train_list, test_list

def get_pie_list(data_idx):
    # list of paths to PIE images
    pie_list = []
    pie_train_list = []
    pie_test_list = []

    for subj_idx in data_idx:
        subj_list = ['PIE/'+str(subj_idx)+'/'+str(i+1)+'.jpg' for i in range(0,NUM_EACH_SUBJECT)]
        subj_train_list, subj_test_list = get_train_test_list(subj_list, ratio=TRAIN_TEST_RATIO, seed=RANDOM_SEED)
        pie_train_list.extend(subj_train_list)
        pie_test_list.extend(subj_test_list)
        pie_list.extend(subj_list)
    return pie_list, pie_train_list, pie_test_list

def get_self_list():
    # list of paths to selfies
    self_list = ['selfimg/'+str(i+1)+'.jpg' for i in range(0,NUM_SELFIES)]
    self_train_list, self_test_list = get_train_test_list(self_list, ratio=TRAIN_TEST_RATIO, seed=RANDOM_SEED)
    return self_list, self_train_list, self_test_list

def get_img(input_list):
    img_v = []
    labels = []
    for i in range(len(input_list)):
        path = input_list[i]
        pathsplit = path.split('/')
        img = cv2.imread(path)
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img_v.append(img)
        if pathsplit[0] == 'PIE':
            labels.append(int(pathsplit[1]))
        elif pathsplit[0] == 'selfimg':
            #labels.append(pathsplit[1]) # label of selfimg is set as 'selfimg'
            labels.append(0) # label of selfimg is set as 0
        else:
            print('Error: Wrong path list!')
        
    img_a = np.array(img_v)
    #img_a = img_a.reshape(len(img_v), -1)
    
    labels_a = np.array(labels)
    
    return img_a, labels_a

#============================================================
#data_idx = choose_random_idx(num=25, vmin=1, vmax=68, seed=RANDOM_SEED)
data_idx = [i for i in range(1, 21)]

pie_list, pie_train_list, pie_test_list = get_pie_list(data_idx)
self_list, self_train_list, self_test_list = get_self_list()

# list of paths to all images of interest
list_img = pie_list + self_list
train_list = pie_train_list + self_train_list
test_list = pie_test_list + self_test_list


print('data_idx',data_idx)
print('Number of PIE images:', len(pie_list))
print('Number of PIE train images:', len(pie_train_list))
print('Number of PIE test images:', len(pie_test_list))
print('Number of self images:', len(self_list))
print('Number of self train images:', len(self_train_list))
print('Number of self test images:', len(self_test_list))
print('Number of whole train images:', len(train_list))
print('Number of whole test images:', len(test_list))

data_idx [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Number of PIE images: 3400
Number of PIE train images: 2380
Number of PIE test images: 1020
Number of self images: 10
Number of self train images: 7
Number of self test images: 3
Number of whole train images: 2387
Number of whole test images: 1023


# CNN

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset, IterableDataset




In [9]:

''' Model '''
print("Model")
print("...")

#构建网络
class ConvolutionNet(nn.Module):
    def __init__(self):
        super(ConvolutionNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size = 5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size = 5)
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(500, 21) 

    def forward(self, x):
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1)
        print(x.size())
        x = self.fc(x)
        return x


print("Model complicated")
model = ConvolutionNet()
print(model)

Model
...
Model complicated
ConvolutionNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=500, out_features=21, bias=True)
)


In [10]:
#构建损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [11]:
train_data, train_label = get_img(train_list)
test_data, test_label = get_img(test_list)

In [12]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])

#首先定义一个Dataset的子类->myDataset
class myDataset(Dataset):
    def __init__(self, name, base_path):
        f = open(name)
        self.filenames = f.readlines()
        f.close()
 #override这两个方法
    def __getitem__(self, index):
        path = self.filenames[index]
        print(os.path.join(base_path, path))
        img = cv2.imread(os.path.join(base_path, path).strip())
        img = torch.Tensor(img)
        return img
    def __len__(self):
        return len(self.filenames)
dataset = myDataset
train_loader = DataLoader(dataset(name=name, base_path=base_path), 
    batch_size=4, shuffle=True)
for img in train_loader:
    print(img.size())
    cv2.imshow('we', np.uint8(img.numpy()[0]))
    cv2.waitKey()

# train_set = TensorDataset(train_data, train_label)
# train_loader = DataLoader(train_set, batch_size=16)

ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

In [9]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss:%.3f' % (epoch + 1, batch_idx + 1, running_loss / 2000))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim = 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print('Accuracy on test set:%d %% [%d/%d]' % (100 * correct / total, correct, total))
    return 100 * correct / total

if __name__ == '__main__':
    len = 10
    x = [0] * len
    y = [0] * len
    for epoch in range(10):
        train(epoch)
        y[epoch] = test()
        x[epoch] = epoch
    plt.plot(x, y)
    plt.xlabel('epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.show()


==> Preparing data..

Epoch: 0


RuntimeError: Given groups=1, weight of size [20, 1, 5, 5], expected input[10, 32, 32, 3] to have 1 channels, but got 32 channels instead